In [1]:
import numpy as np
from itertools import product
import pandas as pd

np.set_printoptions(formatter={'int': '{:+}'.format})

In [2]:
k = 1

In [3]:
def energy(config):
    H = 0
    for i in range(len(config)):
        H += config[i-1] * config[i]

    return -H

In [4]:
def partition_function(configuarations, T):
    Z = 0
    for config in configuarations:
        Z += np.exp(- energy(config) / (k * T))
    
    return Z

In [5]:
def config_probabilities(configuarations, T):
    Z = partition_function(configuarations, T)

    probabilities = np.empty(len(configuarations))
    for config in configuarations:
        probabilities[i] = np.exp(- energy(config) / (k *T)) / Z

    return probabilities

In [6]:
def average_energy(configuarations, T):
    probabilities = config_probabilities(configuarations, T)
    
    E = 0
    for i in range(len(configuarations)):
         E += energy(configuarations[i]) * probabilities[i]
    
    return E

In [7]:
def absolute_magnetisation(configuarations):
    M = np.empty(len(configuarations))

    for i in range(len(configuarations)):
        M[i] = np.sum(configuarations[i]) / len(configuarations)
    
    return M

In [8]:
possible_spins = [+1, -1]
lattice_points = 4

configuarations = np.array(list(product(possible_spins, repeat=lattice_points)))

In [9]:
for i in range(len(configuarations)):
    print(configuarations[i], "\t", energy(configuarations[i]))

[+1 +1 +1 +1] 	 -4
[+1 +1 +1 -1] 	 0
[+1 +1 -1 +1] 	 0
[+1 +1 -1 -1] 	 0
[+1 -1 +1 +1] 	 0
[+1 -1 +1 -1] 	 4
[+1 -1 -1 +1] 	 0
[+1 -1 -1 -1] 	 0
[-1 +1 +1 +1] 	 0
[-1 +1 +1 -1] 	 0
[-1 +1 -1 +1] 	 4
[-1 +1 -1 -1] 	 0
[-1 -1 +1 +1] 	 0
[-1 -1 +1 -1] 	 0
[-1 -1 -1 +1] 	 0
[-1 -1 -1 -1] 	 -4


In [10]:
T = 0.01

print(partition_function(configuarations, T))

1.0442939379528289e+174


In [11]:
probabilities = config_probabilities(configuarations, T)

for i in range(len(configuarations)):
    print(configuarations[i], "\t", probabilities[i])

[+1 +1 +1 +1] 	 4.9695046796555e-310
[+1 +1 +1 -1] 	 0.0
[+1 +1 -1 +1] 	 2.41907520228e-312
[+1 +1 -1 -1] 	 2.143215749364e-312
[+1 -1 +1 +1] 	 2.46151511768e-312
[+1 -1 +1 -1] 	 2.31297541265e-312
[+1 -1 -1 +1] 	 2.355415328525e-312
[+1 -1 -1 -1] 	 2.0583359178e-312
[-1 +1 +1 +1] 	 2.228095581077e-312
[-1 +1 +1 -1] 	 2.5676149076e-312
[-1 +1 -1 +1] 	 2.482735075587e-312
[-1 +1 -1 -1] 	 2.05833591778e-312
[-1 -1 +1 +1] 	 2.05833591782e-312
[-1 -1 +1 -1] 	 2.29175545476e-312
[-1 -1 -1 +1] 	 2.079555875625e-312
[-1 -1 -1 -1] 	 0.5
